In [1]:
import pandas as pd
from data import load_dataset, punctuation
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import modeling

In [2]:
from tokenizers import BertWordPieceTokenizer

# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device = {device}')

# load tokenizer
tokenizer = BertWordPieceTokenizer('./pretrained_tokenizer/vocab.txt', lowercase=False)

device = cuda


In [135]:
df = pd.read_csv('labeled.csv')
p_df = pd.read_csv('pseudo_labeled2.csv')
test_df = pd.read_csv('test_.csv')

df = punctuation(df)

start puncutation


  0%|          | 0/19511 [00:00<?, ?it/s]/opt/ml/project/final-project-level3-nlp-10/data.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['text'][i] = ''.join(text)
100%|██████████| 19511/19511 [00:01<00:00, 10980.36it/s]


In [136]:
true_label = df[df['label']==1]
false_label = df[df['label']==0].sample(frac=0.3)
len(true_label), len(false_label)

(4647, 4459)

In [137]:
true_label = true_label.reset_index().drop(['index'],axis=1)
false_label = false_label.reset_index().drop(['index'],axis=1)

In [138]:
df = true_label.append(false_label)
labels = list(df['label'])

In [139]:
p_true_label = p_df[p_df['label']==1]
p_false_label = p_df[p_df['label']==0].sample(frac=0.1)
len(p_true_label), len(p_false_label)

(14920, 16341)

In [140]:
p_true_label = p_true_label.reset_index().drop(['index'],axis=1)
p_false_label = p_false_label.reset_index().drop(['index'],axis=1)

In [141]:
p_df = p_true_label.append(p_false_label)
p_labels = list(p_df['label'])

In [142]:
test_labels = test_df['label']

In [143]:
def tokenized_sentence(tokenizer, df):
    tokenized = []
    for text in list(df['text']):
        tokens = tokenizer.encode(text).ids
        if len(tokens) <= 200:
            for i in range(200-len(tokens)): tokens.append(0)
        elif len(tokens) > 200:
            for i in range(len(tokens)-200): tokens.pop()
        tokenized.append(tokens)
    return tokenized

In [144]:
print('----labeled data tokenizing-----')
df = tokenized_sentence(tokenizer, df)

print('----unlabeled data tokenizing-----')
p_df = tokenized_sentence(tokenizer, p_df)

print('----test data tokenizing-----')
test_df = tokenized_sentence(tokenizer, test_df)


dataset = load_dataset(df, labels)
p_dataset = load_dataset(p_df, p_labels)
test_dataset = load_dataset(test_df, test_labels)


dataloader = DataLoader(dataset, batch_size=128, shuffle=True)
p_dataloader = DataLoader(p_dataset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)
print('====Finish====')

----labeled data tokenizing-----
----unlabeled data tokenizing-----
----test data tokenizing-----
====Finish====


In [145]:
print(df[0], labels[0])

[2, 12, 14112, 36, 65, 24149, 18981, 65, 42292, 13, 42307, 4979, 5182, 284346, 24866, 4955, 5573, 6180, 5083, 11737, 26618, 5016, 317188, 126904, 20078, 5012, 5426, 12011, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 1


In [146]:
dataset[0]

{'input_ids': tensor([     2,     12,  14112,     36,     65,  24149,  18981,     65,  42292,
             13,  42307,   4979,   5182, 284346,  24866,   4955,   5573,   6180,
           5083,  11737,  26618,   5016, 317188, 126904,  20078,   5012,   5426,
          12011,      3,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0

In [147]:
from utils import Config, set_seed
set_seed(42)

config = Config(
    dropout1=0.3,
    dropout2=0.4,
    learning_rate=0.001,
    label_smoothing=0.5,
    epochs=50,
    embedding_dim=100,
    channel=128)

In [148]:
# Load model
vocab_size = 500000
print(f'vocab size = {vocab_size}')
model = modeling.Model(
    vocab_size=vocab_size, 
    embedding_dim=config.embedding_dim, 
    channel=config.channel, 
    num_class=2,
    dropout1=config.dropout1,
    dropout2=config.dropout2)
model.to(device)

# set criterion, optimizer, schdeuler
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)

vocab size = 500000


/opt/conda/envs/lightweight/lib/python3.7/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
from tqdm import trange

epochs = 100
model.train()
criterion = nn.CrossEntropyLoss()
for epoch in trange(epochs):
    running_loss = 0
    for i, labeled in enumerate(dataloader):
        input = labeled['input_ids'].to(device)
        label = labeled['label'].to(device)

        output = model(input)
        loss = criterion(output, label)
        running_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch: {epoch+1} | Train Loss : {running_loss/len(dataloader):.5f}')
torch.save(model.state_dict(), f'./pseudo/result_100.pt')

In [17]:
def alpha_weight(step):
    # T1, T2 : hyperparameters
    T1 = 100
    T2 = 700
    af = 3.0
    if step < T1:
        return 0.0
    elif step > T2:
        return af
    else:
         return ((step-T1) / (T2-T1))*af

In [24]:
import gc
torch.cuda.empty_cache()
gc.collect()

epochs = 500
step = 100
best_acc = 0.75
model.load_state_dict(torch.load('./pseudo/result_100.pt'))
for epoch in trange(epochs):
    for i, unlabeled in enumerate(p_dataloader):
        input = unlabeled['input_ids'].to(device)
        
        model.eval()
        output_unlabeled = model(input)
        _, pseudo_labeled = torch.max(output_unlabeled, 1)
        
        model.train()
        output = model(input)
        unlabeled_loss = alpha_weight(step) * F.cross_entropy(output, pseudo_labeled)

        optimizer.zero_grad()
        unlabeled_loss.backward()
        optimizer.step()

        if (i+1) % 50 == 0:
            for j, batch in enumerate(dataloader):
                input = batch['input_ids'].to(device)
                label = batch['label'].to(device)
                output = model(input)
                loss = F.cross_entropy(output, label)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
            step += 1
    
    if (epoch+1) % 10 == 0:
        model.eval()
        correct, loss = 0, 0
        zero, one = 0, 0
        with torch.no_grad():
            for i, batch in enumerate(test_dataloader):
                data = batch['input_ids'].cuda()
                labels = batch['label']
                output = model(data)
                predicted = torch.max(output,1)[1]
                
                zero += predicted.tolist().count(0)
                one += predicted.tolist().count(1)
                
                correct += (predicted==labels.cuda()).sum()
                loss += F.cross_entropy(output, labels.cuda()).item()
        
        print(f'Epoch: {epoch+1} | Alpha : {alpha_weight(step)} | Train Loss : {loss/len(test_dataloader):.5f} | Test Acc : {correct/len(test_dataset):.5f} | Zero : {zero} | One : {one}')
        
        eval_acc = correct/len(test_dataloader)
        if eval_acc > best_acc:
            torch.save(model.state_dict(), f'./save/pseudo/result.pt')
            best_acc = eval_acc
        
        model.train()

43it [00:00, 82.75it/s][03:12<2:54:02, 21.27s/it]


Epoch: 10 | Train Loss : 0.01293 | Test Acc : 0.99686 | Zero : 2652 | One : 2755


43it [00:00, 82.55it/s] [06:45<2:50:05, 21.22s/it]


Epoch: 20 | Train Loss : 0.00337 | Test Acc : 0.99963 | Zero : 2665 | One : 2742


43it [00:00, 82.33it/s] [10:19<2:46:57, 21.27s/it]


...

43it [00:00, 82.60it/s]0 [46:01<2:11:29, 21.26s/it]
 26%|██▌       | 130/500 [46:23<2:11:58, 21.40s/it]

Epoch: 130 | Train Loss : 0.00228 | Test Acc : 0.99963 | Zero : 2665 | One : 2742


43it [00:00, 82.57it/s]0 [49:35<2:08:35, 21.37s/it]
 28%|██▊       | 140/500 [49:57<2:08:58, 21.49s/it]

Epoch: 140 | Train Loss : 0.00273 | Test Acc : 0.99963 | Zero : 2665 | One : 2742


 30%|██▉       | 148/500 [52:47<2:05:04, 21.32s/it]

In [ ]:
torch.save(model.state_dict(), './pseudo/result.pt')

### Test

In [107]:
model.load_state_dict(torch.load('./pseudo/result_100.pt'))
model.eval()
model.cuda()

Model(
  (embedding): Embedding(500000, 100)
  (conv1): Conv1d(100, 128, kernel_size=(2,), stride=(1,))
  (conv2): Conv1d(100, 128, kernel_size=(4,), stride=(1,))
  (conv3): Conv1d(100, 128, kernel_size=(6,), stride=(1,))
  (fc): Linear(in_features=384, out_features=128, bias=True)
  (classifier): Linear(in_features=512, out_features=2, bias=True)
  (lstm): LSTM(100, 128, batch_first=True, dropout=0.2, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (dropout2): Dropout(p=0.4, inplace=False)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (AvgPool): AdaptiveAvgPool1d(output_size=128)
)

In [149]:
text = '700'

tokens = tokenizer.encode(text).ids

if len(tokens) <= 200:
    for i in range(200-len(tokens)): tokens.append(0)
elif len(tokens) > 200:
    for i in range(len(tokens)-200): tokens.pop()
print(tokens)

[2, 15587, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [150]:
input = torch.tensor([tokens]).cuda()

output = model(input)

print(F.softmax(output))
print(torch.argmax(output, -1))

tensor([[0.8995, 0.1005]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
tensor([0], device='cuda:0')


/opt/conda/envs/lightweight/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """
